# Requirements

In [ ]:
import json
import os
import numpy as np
from natsort import natsorted

from utility import (
    frames_to_video,
    read_folder_of_jsons,
    flight_analysis,
)

# Prepare flight data for D3VO

- D3VO needs a flight video in .mp4 so we'll need to convert our frames into a video.

In [ ]:
# First, download the flight data...
!mkdir flight004_08_18_2025
!gcloud alpha storage cp \
    gs://tera-earth-dev-users/bolt/18-august-2025/flight_4_8_18_in/* \
    flight004_08_18_2025/
# Download the ground truth csv as well...
!gcloud alpha storage cp \
    gs://tera-earth-dev-users/bolt/18-august-2025/flight004_08_18_2025_ground_truth.csv \
    flight004_08_18_2025_ground_truth.csv

In [ ]:
# stitch frames into a video...
flight = "flight004_08_18_2025"
jpg_files = natsorted([f"{flight}/{f}" for f in os.listdir(flight) if f.endswith(".jpg")])

frames_to_video(jpg_files, f"{flight}.mp4")

In [ ]:
command = f"""python3 src/main.py \\
    "{flight}.mp4" \\
    "weights/" \\
    --save "poses--{flight}.npy"
"""
print(command)
# Now, run this command in the terminal...

# Evaluation

In [ ]:
def write_pred_poses_to_jsons(poses, timestamps, output_dir: str):
    """Write predicted poses to JSON files as tera-system1 convention just so
    it's easy to compare."""
    os.makedirs(output_dir, exist_ok=True)
    for i, pose in enumerate(poses):
        pose_xyz = pose[:3, 3]
        x, y, z = pose_xyz[0], pose_xyz[1], pose_xyz[2]
        yaw = pose[3, 3]
        
        pred_frame_filepath = os.path.join(output_dir, f"pred{i:05d}.json")
        with open(pred_frame_filepath, "w") as f:
            json.dump({
                "frame_filename": f"pred{i:05d}.png",
                "timestamp": timestamps[i],
                "x": x,
                "y": y,
                "z": z,
                "p": yaw,
            }, f, indent=4)

In [ ]:
# Write predicted poses to JSON files...
pred_poses = np.load(f"poses--{flight}.npy")

# fetch timestamps from input frames...
timestamps = [d[1]["timestamp"] for d in read_folder_of_jsons(flight)][:len(pred_poses)]
write_pred_poses_to_jsons(
    pred_poses,
    timestamps,
    f"{flight}--pred"
)

In [ ]:
# Plot predictions & calculate KITTI metrics...
flight_analysis(
    gt_path=f"{flight}_ground_truth.csv",
    pred_folder=f"{flight}--pred",
    output_path=f"{flight}-pred-analysis/",
    save_files=True,
)